<a href="https://colab.research.google.com/github/devroopsaha744/HateSpeechDetect-text/blob/main/Hatespeech_detection_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install datasets -q
!pip install nltk -q
!pip install gensim -q
!pip install imbalanced-learn -q

In [3]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [19]:
import numpy as np
import pandas as pd
from gensim.models import Word2Vec
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

In [5]:
from datasets import load_dataset
data = load_dataset("tdavidson/hate_speech_offensive")

Generating train split:   0%|          | 0/24783 [00:00<?, ? examples/s]

In [6]:
data_split = data['train'].train_test_split(test_size = 0.3)

In [7]:
train = data_split['train']
test = data_split['test']

In [8]:
train

Dataset({
    features: ['count', 'hate_speech_count', 'offensive_language_count', 'neither_count', 'class', 'tweet'],
    num_rows: 17348
})

In [9]:
test

Dataset({
    features: ['count', 'hate_speech_count', 'offensive_language_count', 'neither_count', 'class', 'tweet'],
    num_rows: 7435
})

In [10]:
train_df =  train.to_pandas()
test_df = test.to_pandas()

In [ ]:
train_df

,count,hate_speech_count,offensive_language_count,neither_count,class,tweet
0,6,1,1,4,2,"RT @sorryimalex: ""now 5sos is starting to twee..."
1,3,0,3,0,1,"RT @nandotrash: Tell a girl to ""calm down"" she..."
2,3,0,3,0,1,"RT @thecoreyholcomb: Twitter family, I promise..."
3,6,0,1,5,2,@Notsosweetpea SOO when we hanging out and mak...
4,3,0,3,0,1,"So, #brony-folk far and wide are throwing bitc..."
...,...,...,...,...,...,...
17343,3,0,3,0,1,Nobody wit me im on all alone shit so dont thi...
17344,3,1,2,0,1,@GYPMATIC45 i pissed alotta ppl off on fb i wa...
17345,3,0,3,0,1,Miss ma niggah leo
17346,3,0,3,0,1,"If eating the pussy makes you happy, I'm all f..."


In [12]:
train_df['class'].value_counts()

class
1    13441
2     2888
0     1019
Name: count, dtype: int64

In [13]:
test_df

,count,hate_speech_count,offensive_language_count,neither_count,class,tweet
0,3,0,3,0,1,@Bitcheslovestar The only one of Mary's friend...
1,3,0,3,0,1,No skimp shit here bitch
2,3,0,3,0,1,these bitches singing this song tho -_-
3,3,1,2,0,1,@KodyMark1 hahahh thank you fag boy! #represent
4,3,0,3,0,1,Soon as da money gon dem bitches leavin..You g...
...,...,...,...,...,...,...
7430,3,0,0,3,2,@AustinBurge44 either way both schools are tra...
7431,3,0,3,0,1,@jballinbutts420 sameeee bitch asss lol
7432,3,0,3,0,1,"@n0tsteven ily 2, I'm at work bitch do you eve..."
7433,3,1,2,0,1,Everyone that says #LetsMakeItAwkward is a que...


In [14]:
# Load the stop words
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    # Lowercase the text
    text = text.lower()
    # Remove special characters and punctuation
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    # Tokenize the text
    tokens = word_tokenize(text)
    # Remove stop words and lemmatize the tokens
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    # Join the tokens back into a single string
    text = ' '.join(tokens)
    return text

# Apply the preprocessing function to the 'text' column
train_df['tweet'] = train_df['tweet'].apply(preprocess_text)

In [15]:
test_df['tweet'] = test_df['tweet'].apply(preprocess_text)

In [ ]:
train_df

,count,hate_speech_count,offensive_language_count,neither_count,class,tweet
0,6,1,1,4,2,rt sorryimalex 5sos starting tweet hashtag the...
1,3,0,3,0,1,rt nandotrash tell girl calm shell act like ki...
2,3,0,3,0,1,rt thecoreyholcomb twitter family promise put ...
3,6,0,1,5,2,notsosweetpea soo hanging makin brownie
4,3,0,3,0,1,bronyfolk far wide throwing bitch fit equestri...
...,...,...,...,...,...,...
17343,3,0,3,0,1,nobody wit im alone shit dont think im actin f...
17344,3,1,2,0,1,gypmatic45 pissed alotta ppl fb flooding newsf...
17345,3,0,3,0,1,miss niggah leo
17346,3,0,3,0,1,eating pussy make happy im


In [16]:
train_df['tokens'] = train_df['tweet'].apply(word_tokenize)

# Train Word2Vec model
model = Word2Vec(sentences=train_df['tokens'], vector_size=100, window=5, min_count=1, workers=4)

# Save the model for future use
model.save("word2vec.model")

# Load the model
# model = Word2Vec.load("word2vec.model")


In [17]:
test_df['tokens'] = test_df['tweet'].apply(word_tokenize)

In [18]:
def get_average_word2vec(tokens, model, vector_size):
    # Initialize an empty vector
    vec = np.zeros(vector_size)
    count = 0
    for word in tokens:
        if word in model.wv:
            vec += model.wv[word]
            count += 1
    if count != 0:
        vec /= count
    return vec

vector_size = model.vector_size

train_df['text_vec'] = train_df['tokens'].apply(lambda x: get_average_word2vec(x, model, vector_size))
test_df['text_vec'] = test_df['tokens'].apply(lambda x: get_average_word2vec(x, model, vector_size))

# Convert lists of vectors to a 2D array for the classifier
X_train = np.stack(train_df['text_vec'].values)
y_train = train_df['class'].values
X_test = np.stack(test_df['text_vec'].values)
y_test = test_df['class'].values


In [21]:
#Applying data-resampling
# Define the resampling pipeline
over = SMOTE(sampling_strategy= 'auto')  # Oversample the minority class to 10% of the majority class
under = RandomUnderSampler(sampling_strategy='auto')  # Undersample the majority class to 50% of the original

# Combine the oversampling and undersampling in a pipeline
steps = [('o', over), ('u', under)]
pipeline = Pipeline(steps=steps)

# Apply resampling to the training data
X_train_res, y_train_res = pipeline.fit_resample(X_train, y_train)

In [ ]:
#On original Data
clf1 = SVC()
clf2 = LogisticRegression()
clf3 = DecisionTreeClassifier()
clf4 = RandomForestClassifier()
clf5 = GradientBoostingClassifier()
clf6 = XGBClassifier()
clf7 = AdaBoostClassifier()

classifier = [clf1, clf2, clf3, clf4, clf5, clf6, clf7]
for clf in classifier:
  clf.fit(X_train, y_train)
  y_pred = clf.predict(X_test)
  print(f"The Classification report of {clf} is: \n {classification_report(y_test, y_pred)}")
  print("\n")

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


The Classification report of SVC() is: 
               precision    recall  f1-score   support

           0       0.00      0.00      0.00       438
           1       0.78      0.99      0.87      5681
           2       0.63      0.12      0.20      1316

    accuracy                           0.78      7435
   macro avg       0.47      0.37      0.36      7435
weighted avg       0.71      0.78      0.70      7435





/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels wit

The Classification report of LogisticRegression() is: 
               precision    recall  f1-score   support

           0       0.00      0.00      0.00       438
           1       0.84      0.97      0.90      5681
           2       0.71      0.49      0.58      1316

    accuracy                           0.83      7435
   macro avg       0.52      0.49      0.49      7435
weighted avg       0.77      0.83      0.79      7435



The Classification report of DecisionTreeClassifier() is: 
               precision    recall  f1-score   support

           0       0.12      0.13      0.12       438
           1       0.87      0.88      0.87      5681
           2       0.58      0.53      0.55      1316

    accuracy                           0.77      7435
   macro avg       0.52      0.51      0.52      7435
weighted avg       0.77      0.77      0.77      7435



The Classification report of RandomForestClassifier() is: 
               precision    recall  f1-score   support

   

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [23]:
#On resampled Data
clf1 = SVC()
clf2 = LogisticRegression()
clf3 = DecisionTreeClassifier()
clf4 = RandomForestClassifier()
clf5 = GradientBoostingClassifier()
clf6 = XGBClassifier()
clf7 = AdaBoostClassifier()

classifier = [clf1, clf2, clf3, clf4, clf5, clf6, clf7]
for clf in classifier:
  clf.fit(X_train_res, y_train_res)
  y_pred = clf.predict(X_test)
  print(f"The Classification report of {clf} is: \n {classification_report(y_test, y_pred)}")
  print("\n")

The Classification report of SVC() is: 
               precision    recall  f1-score   support

           0       0.12      0.26      0.17       411
           1       0.94      0.79      0.86      5749
           2       0.55      0.73      0.63      1275

    accuracy                           0.75      7435
   macro avg       0.54      0.60      0.55      7435
weighted avg       0.83      0.75      0.78      7435





/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


The Classification report of LogisticRegression() is: 
               precision    recall  f1-score   support

           0       0.15      0.48      0.23       411
           1       0.97      0.76      0.85      5749
           2       0.62      0.78      0.69      1275

    accuracy                           0.75      7435
   macro avg       0.58      0.67      0.59      7435
weighted avg       0.86      0.75      0.79      7435



The Classification report of DecisionTreeClassifier() is: 
               precision    recall  f1-score   support

           0       0.10      0.25      0.14       411
           1       0.89      0.79      0.84      5749
           2       0.56      0.55      0.56      1275

    accuracy                           0.72      7435
   macro avg       0.51      0.53      0.51      7435
weighted avg       0.79      0.72      0.75      7435



The Classification report of RandomForestClassifier() is: 
               precision    recall  f1-score   support

   